In [3]:

import os
import pandas as pd
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings


In [4]:

# === CONFIG ===
CSV_PATH = "./data/job_descriptions_large.csv"
CHROMA_DIR = "./chroma_db"


In [7]:

# === STEP 1: Prepare Vector Store ===
if not os.path.exists(CHROMA_DIR):
    print("📦 Creating new Chroma vector DB...")
    df = pd.read_csv(CSV_PATH)

    # Convert rows to LangChain Documents
    documents = [
        Document(page_content=row["Description"], metadata={"title": row["Job Title"]})
        for _, row in df.iterrows()
    ]

    embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vector_db = Chroma.from_documents(documents, embedding=embedding, persist_directory=CHROMA_DIR)
    vector_db.persist()
else:
    print("✅ Loading existing Chroma vector DB...")
    embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vector_db = Chroma(persist_directory=CHROMA_DIR, embedding_function=embedding)


✅ Loading existing Chroma vector DB...


In [8]:

# === STEP 2: Setup LLM & Prompt ===
llm = Ollama(model="llama3")

prompt_template = PromptTemplate(
    input_variables=["traits", "context"],
    template="""
You are an intelligent and concise career advisor.

A candidate has shared these personality traits:
{traits}

Below are job role descriptions you may use as reference:
{context}

From the roles below, pick the single **best fit**:
- Software Engineer (SE)
- Quality Assurance Engineer (QA)
- Business Analyst (BA)

Your response must include:
1. The best-fit role (SE, QA, or BA)
2. 2-3 traits from the input that support the match
3. A brief explanation in 2–3 sentences

Respond in this exact format:

[Recommended Role]: <SE / QA / BA>
[Supporting Traits]: <trait1>, <trait2>, ...
[Reasoning]: <short explanation>
"""
)


chain = LLMChain(llm=llm, prompt=prompt_template)


C:\Users\Buddima Dissanayake\AppData\Local\Temp\ipykernel_4044\1143978823.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3")
C:\Users\Buddima Dissanayake\AppData\Local\Temp\ipykernel_4044\1143978823.py:34: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)


In [9]:

# === STEP 3: Conversational Loop ===
print("\n🧠 Job Recommendation Chatbot (type 'exit' to quit)\n")

while True:
    user_input = input("🧑‍💼 Describe your traits, behaviors, or preferences: ")
    if user_input.lower() == "exit":
        print("👋 Goodbye!")
        break

    # Get similar job docs from vector DB
    docs = vector_db.similarity_search(user_input, k=5)
    context = "\n\n".join([doc.page_content for doc in docs])

    try:
        result = chain.run(traits=user_input, context=context)
        print("\n🤖 Recommendation:\n", result)
        print("-" * 60)
    except Exception as e:
        print("⚠️ Error:", e)



🧠 Job Recommendation Chatbot (type 'exit' to quit)



C:\Users\Buddima Dissanayake\AppData\Local\Temp\ipykernel_4044\1274253766.py:15: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain.run(traits=user_input, context=context)



🤖 Recommendation:
 **Best-Fit Role:** Business Analyst (BA)
**Supporting Traits:** I love to problem solve, Analysis doctor store certain very, Deep list then meet natural production professor eye.
**Reasoning:** The role of a Business Analyst involves analyzing complex business problems and developing solutions. With the candidate's love for problem-solving and analytical skills, this role is an excellent fit. Additionally, BA requires communication with various stakeholders to discuss findings and recommendations, which aligns with the candidate's ability to help others and meet their needs.
------------------------------------------------------------


KeyboardInterrupt: 